## IMPORT NECESSARY LIBRARIES

In [ ]:
import os
import cv2
from imutils import paths
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import models,layers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings('ignore')




In [ ]:

#Change runtime type

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
datapath ="/content/gdrive/MyDrive/Sign pics/Sign pics"

## READ DATA

In [ ]:
pathToImage =list(paths.list_images(datapath))
#print(pathToImage)
data =[]
labels =[]
for images in pathToImage:
    label=images.split(os.path.sep)[-2]
    #print(label)
    image =cv2.imread(images)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    data.append(image)
    labels.append(label)
    #plt.figure()


In [ ]:
#print(labels)

In [ ]:
# Total number of images and labels
print('Total number of images',len(data))
print('Total number of labels',len(set(labels)))
print("Labels: ",[X for X in set(labels)])

In [ ]:
#plt.figure(figsize =(20,20))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(data[i])
    plt.title(labels[i])
    plt.axis('off')

In [ ]:
blur_img =[]
clear_image =[]
for img in data:
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    Gaussain_blur =cv2.GaussianBlur(gray,(5,5),2) 
    th3=cv2.adaptiveThreshold(Gaussain_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret,res = cv2.threshold(th3, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(res, cv2.MORPH_CLOSE, kernel, iterations=1)
    #erosion = cv2.erode(closing, kernel, iterations=1)
    clear_image.append(closing)       
    blur_img.append(Gaussain_blur)
    

In [ ]:
print(ret)

In [ ]:
print(res)

In [ ]:
plt.imshow(gray)

In [ ]:
plt.imshow(clear_image[1])

In [ ]:
clear_image[1]

## Gaussian blur image

In [ ]:
#plt.figure(figsize =(20,20))
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(blur_img[i])
    plt.title(labels[i])
    plt.axis('off')
    
    
    

## clear image

In [ ]:
plt.figure(figsize =(30,30))
for i in range(250):
    plt.subplot(25, 10, i + 1)
    plt.imshow(clear_image[i],cmap ="gray")
    plt.title(labels[i])
    plt.axis('off')

In [ ]:
data =np.array(clear_image)
labels =np.array(labels)


## SPLITTING DATA 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,labels,test_size =0.2,stratify =labels,random_state =42)

## SHAPE OF TRAINING AND TESTING DATA

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

## NORMALIZING THE DATA

In [ ]:
#x_train =Normalizer().fit_transform(x_train)
#x_test =Normalizer().fit_transform(x_test)

In [ ]:
x_train =x_train/255
x_test = x_test/255

In [ ]:
print(x_train)

In [ ]:
print(y_train)

In [ ]:
#x_train = x_train.reshape(120, 224*224*3)
#x_test = x_test.reshape(30,224*224*3 )

In [ ]:
x_train = x_train.reshape(962,224,224,1)
x_test =x_test.reshape(241,224,224,1)

In [ ]:
x_test.shape

In [ ]:
x_train.ndim

In [ ]:
y_train

In [ ]:
#x_train.reshape()

In [ ]:
#x_train =Normalizer().fit_transform(x_train)
#x_test =Normalizer().fit_transform(x_test)

In [ ]:
x_train[0].max()

In [ ]:
lb = preprocessing.LabelBinarizer()
y_train =lb.fit_transform(y_train)
y_test =lb.fit_transform(y_test)

In [ ]:
x_test.shape,y_test.shape

## DATA GENERATOR

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically 
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )  
datagen.fit(x_train)

## CNN MODEL

In [ ]:
#CNN
# 1st sequential for forward and backoard propogation
model = models.Sequential()
#adding convolution layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1),padding ='same'))
#max pooling
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu',padding ='same'))
model.add(layers.MaxPooling2D((2, 2)))
# Drop_out
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding ='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(128, (3, 3), activation='relu',padding ='same'))
model.add(layers.MaxPooling2D((2, 2)))


In [ ]:
# now flatten 
model.add(layers.Flatten())
# fully connected neural network
model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dense(200,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_test.shape,y_test.shape

In [ ]:
y_train

In [ ]:
"""# early stopping
import tensorflow as tf
early_stopping =tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)"""

In [ ]:
# compile and train model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(datagen.flow(x_train, y_train), epochs = 100, validation_data=(x_test,y_test))

## PLOTTING 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

#test_loss, test_acc = model.evaluate(validation_ds, verbose=2)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('val_loss')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
score = model.evaluate(x_test, y_test,batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_predict=model.predict(x_test)
print(y_predict)
y_result  =np.argmax(y_predict,axis =1)
print(y_result)
y_test=np.argmax(y_test,axis =1)

In [ ]:
print(y_result)

In [ ]:
#plt.imshow(x_test[0],cmap="gray")

In [ ]:
model.save('cap_keras_model.h5')

## PLOTTING CONFUSION MATRIX

In [ ]:
from sklearn.metrics import classification_report as cr,accuracy_score,confusion_matrix
cm =confusion_matrix(y_test,y_result)
sns.heatmap(cm,annot =True)
print("accuracy_score",accuracy_score(y_test,y_result))
print()
print('confusion_matrix',cr(y_test,y_result))

In [ ]:
y_test

# PREDICTION

In [ ]:
image =cv2.imread("/content/gdrive/MyDrive/TEST_DATA/A-samples/4.jpg")
gray =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
Gaussain_blur =cv2.GaussianBlur(gray,(5,5),4)
th3=cv2.adaptiveThreshold(Gaussain_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
ret,res = cv2.threshold(th3, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3, 3), np.uint8)
closing = cv2.morphologyEx(res, cv2.MORPH_CLOSE, kernel, iterations=1)
test_image =closing/255
test_image = np.expand_dims(test_image,0)
test_image = test_image.reshape(1,224,224,1)


plt.imshow(res,cmap ="gray")
print(test_image.shape)

In [ ]:
pred_img=model.predict(np.array(test_image))
print(pred_img)
pred_result  =np.argmax(pred_img,axis =1)
pred_result

In [ ]:
image_b =cv2.imread('/content/gdrive/MyDrive/TEST_DATA/B -samples/11.jpg')
gray =cv2.cvtColor(image_b,cv2.COLOR_BGR2GRAY)
Gaussain_blur =cv2.GaussianBlur(gray,(5,5),4)
th3=cv2.adaptiveThreshold(Gaussain_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
ret,res = cv2.threshold(th3, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3, 3), np.uint8)
closing = cv2.morphologyEx(res, cv2.MORPH_CLOSE, kernel, iterations=1)
test_image =closing/255
test_image = np.expand_dims(test_image,0)
test_image = test_image.reshape(1,224,224,1)
plt.imshow(res,cmap ="gray")
print(test_image.shape)

In [ ]:
pred_img=model.predict(np.array(test_image))
print(pred_img)
pred_result  =np.argmax(pred_img,axis =1)
pred_result

In [ ]:
image_c =cv2.imread("/content/gdrive/MyDrive/TEST_DATA/C-samples/8.jpg")
gray =cv2.cvtColor(image_c,cv2.COLOR_BGR2GRAY)
Gaussain_blur =cv2.GaussianBlur(gray,(5,5),4)
th3=cv2.adaptiveThreshold(Gaussain_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
ret,res = cv2.threshold(th3, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3, 3), np.uint8)
closing = cv2.morphologyEx(res, cv2.MORPH_CLOSE, kernel, iterations=1)
test_image =closing/255
test_image = np.expand_dims(test_image,0)
test_image = test_image.reshape(1,224,224,1)
plt.imshow(res,cmap ="gray")
print(test_image.shape)

In [ ]:
pred_img=model.predict(np.array(test_image))
print(pred_img)
pred_result  =np.argmax(pred_img,axis =1)
pred_result